# Opening 6.5km MAR Netcdfs

opening the NCs that were uploaded to ldeo-glaciology. See [this](https://github.com/ldeo-glaciology/pangeo-MAR/issues/4) issue for details. 

This method DOES work

In [13]:
import gcsfs
import xarray as xr
import hvplot.xarray
 
gcs = gcsfs.GCSFileSystem()

url = 'gs://ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2000.nc'  
#url = 'gs://ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2001.nc'  


of = gcs.open(url, mode='rb')
ds = xr.open_dataset(of,chunks=-1)    # need the chunks arguement to make it load as dask xarray
print(f"Total size of this year of data: {ds.nbytes/1e9} Gb")

## plotting
### use hvplot as its makes nice interactive plots
options = dict(x='x', y='y', clim=(0.2, 1), width=500, height=700, cmap='greys_r')

ds.AL2.hvplot(**options)

Size: 14.075068657 Gb


:DynamicMap   [SECTOR1_1,TIME]
   :Image   [x,y]   (AL2)

## Below here are two methods that I tried that didnt work for two different reasons. 

This method DOES NOT work, because it says 
```
ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib', 'pydap', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies,
```

In [8]:
import fsspec
import xarray as xr

mapper = fsspec.get_mapper('gs://ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2000.nc', mode='ab')
ds = xr.open_dataset(mapper,chunks={'x': -1, 'y': -1, 'TIME': -1}) 

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib', 'pydap', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

This method DOES work, but causes problems because it closes the file afterwards and you cant access it. Also if you try to rerun the cell you get an 'i/o on closed file' error. 

In [2]:
import gcsfs
import xarray as xr
xr.set_options(display_style="html")

gcs = gcsfs.GCSFileSystem()

url = 'gs://ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2000.nc'  
with gcs.open(url, mode='rb') as of:
    ds = xr.open_dataset(of,chunks={'x': -1, 'y': -1, 'TIME': -1})
    ds.load()
    print(f"{ds.nbytes/1e6} megabytes")
    

KeyboardInterrupt: 